In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Live Trading Results Analysis\n",
    "\n",
    "Analyze real trading performance on Binance testnet"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sys\n",
    "sys.path.append('../python')\n",
    "\n",
    "import json\n",
    "import numpy as np\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from pathlib import Path\n",
    "from datetime import datetime\n",
    "\n",
    "sns.set_style('whitegrid')\n",
    "plt.rcParams['figure.figsize'] = (14, 6)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Load Live Trading Logs"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Find log files\n",
    "log_dir = Path('../data/logs')\n",
    "log_files = list(log_dir.glob('*.json'))\n",
    "\n",
    "print(f\"Found {len(log_files)} trading sessions:\\n\")\n",
    "for log_file in sorted(log_files):\n",
    "    print(f\"  - {log_file.name}\")\n",
    "\n",
    "# Load most recent\n",
    "if log_files:\n",
    "    latest_log = sorted(log_files)[-1]\n",
    "    print(f\"\\nAnalyzing: {latest_log.name}\")\n",
    "    \n",
    "    with open(latest_log, 'r') as f:\n",
    "        data = json.load(f)\n",
    "    \n",
    "    metadata = data['metadata']\n",
    "    df = pd.DataFrame(data['steps'])\n",
    "    \n",
    "    print(f\"\\nSession Info:\")\n",
    "    print(f\"  Symbol: {metadata['symbol']}\")\n",
    "    print(f\"  Model: {metadata['model_type']}\")\n",
    "    print(f\"  Start: {metadata['start_time']}\")\n",
    "    print(f\"  Duration: {len(df)} steps\")\n",
    "else:\n",
    "    print(\"\\n❌ No log files found. Run live trading first!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Performance Overview"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if log_files:\n",
    "    print(\"=\"*60)\n",
    "    print(\"LIVE TRADING PERFORMANCE\")\n",
    "    print(\"=\"*60)\n",
    "    print(f\"Final PnL: ${df['pnl'].iloc[-1]:,.2f}\")\n",
    "    print(f\"Max PnL: ${df['pnl'].max():,.2f}\")\n",
    "    print(f\"Min PnL: ${df['pnl'].min():,.2f}\")\n",
    "    print(f\"Price Range: ${df['mid_price'].min():,.2f} - ${df['mid_price'].max():,.2f}\")\n",
    "    print(f\"Final Inventory: {df['inventory'].iloc[-1]:+.6f} BTC\")\n",
    "    print(f\"Max Inventory: {df['inventory'].abs().max():.6f} BTC\")\n",
    "    print(f\"Avg Spread: {df['spread_bps'].mean():.2f} bps\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. PnL Evolution"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if log_files:\n",
    "    fig, axes = plt.subplots(2, 1, figsize=(14, 10))\n",
    "    \n",
    "    # PnL over time\n",
    "    axes[0].plot(df.index, df['pnl'], linewidth=2, color='green')\n",
    "    axes[0].fill_between(df.index, df['pnl'], 0, alpha=0.3, color='green')\n",
    "    axes[0].axhline(y=0, color='r', linestyle='--', alpha=0.5)\n",
    "    axes[0].set_title('Live Trading PnL Evolution', fontsize=14, fontweight='bold')\n",
    "    axes[0].set_xlabel('Step')\n",
    "    axes[0].set_ylabel('PnL ($)')\n",
    "    axes[0].grid(True, alpha=0.3)\n",
    "    \n",
    "    # Add annotations for key points\n",
    "    max_pnl_idx = df['pnl'].idxmax()\n",
    "    min_pnl_idx = df['pnl'].idxmin()\n",
    "    axes[0].scatter([max_pnl_idx], [df['pnl'].iloc[max_pnl_idx]], \n",
    "                    color='green', s=100, zorder=5, label=f\"Max: ${df['pnl'].max():.2f}\")\n",
    "    axes[0].scatter([min_pnl_idx], [df['pnl'].iloc[min_pnl_idx]], \n",
    "                    color='red', s=100, zorder=5, label=f\"Min: ${df['pnl'].min():.2f}\")\n",
    "    axes[0].legend()\n",
    "    \n",
    "    # Drawdown\n",
    "    running_max = df['pnl'].cummax()\n",
    "    drawdown = (df['pnl'] - running_max)\n",
    "    axes[1].fill_between(df.index, drawdown, 0, color='red', alpha=0.3)\n",
    "    axes[1].plot(df.index, drawdown, color='darkred', linewidth=2)\n",
    "    axes[1].set_title('Drawdown', fontsize=14, fontweight='bold')\n",
    "    axes[1].set_xlabel('Step')\n",
    "    axes[1].set_ylabel('Drawdown ($)')\n",
    "    axes[1].grid(True, alpha=0.3)\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "    \n",
    "    max_dd = drawdown.min()\n",
    "    print(f\"\\nMax Drawdown: ${max_dd:.2f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Inventory Management"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if log_files:\n",
    "    fig, axes = plt.subplots(2, 1, figsize=(14, 10))\n",
    "    \n",
    "    # Inventory over time\n",
    "    axes[0].plot(df.index, df['inventory'], linewidth=2, color='orange')\n",
    "    axes[0].axhline(y=0, color='r', linestyle='--', alpha=0.5)\n",
    "    axes[0].fill_between(df.index, df['inventory'], 0, alpha=0.3, color='orange')\n",
    "    axes[0].set_title('Inventory Position Over Time', fontsize=14, fontweight='bold')\n",
    "    axes[0].set_xlabel('Step')\n",
    "    axes[0].set_ylabel('Inventory (BTC)')\n",
    "    axes[0].grid(True, alpha=0.3)\n",
    "    \n",
    "    # Inventory histogram\n",
    "    axes[1].hist(df['inventory'], bins=30, alpha=0.7, edgecolor='black', color='orange')\n",
    "    axes[1].axvline(x=0, color='r', linestyle='--', linewidth=2, label='Neutral')\n",
    "    axes[1].axvline(x=df['inventory'].mean(), color='g', linestyle='--', linewidth=2, \n",
    "                    label=f\"Mean: {df['inventory'].mean():.6f}\")\n",
    "    axes[1].set_title('Inventory Distribution', fontsize=14, fontweight='bold')\n",
    "    axes[1].set_xlabel('Inventory (BTC)')\n",
    "    axes[1].set_ylabel('Frequency')\n",
    "    axes[1].legend()\n",
    "    axes[1].grid(True, alpha=0.3)\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Market Conditions"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if log_files:\n",
    "    fig, axes = plt.subplots(2, 1, figsize=(14, 10))\n",
    "    \n",
    "    # Price evolution\n",
    "    axes[0].plot(df.index, df['mid_price'], linewidth=2, color='blue')\n",
    "    axes[0].set_title('BTC Price During Trading Session', fontsize=14, fontweight='bold')\n",
    "    axes[0].set_xlabel('Step')\n",
    "    axes[0].set_ylabel('Price ($)')\n",
    "    axes[0].grid(True, alpha=0.3)\n",
    "    \n",
    "    # Spread evolution\n",
    "    axes[1].plot(df.index, df['spread_bps'], linewidth=2, color='purple')\n",
    "    axes[1].axhline(y=df['spread_bps'].mean(), color='r', linestyle='--', \n",
    "                    label=f\"Mean: {df['spread_bps'].mean():.2f} bps\")\n",
    "    axes[1].set_title('Market Spread Over Time', fontsize=14, fontweight='bold')\n",
    "    axes[1].set_xlabel('Step')\n",
    "    axes[1].set_ylabel('Spread (bps)')\n",
    "    axes[1].legend()\n",
    "    axes[1].grid(True, alpha=0.3)\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "    \n",
    "    # Price statistics\n",
    "    price_change = (df['mid_price'].iloc[-1] / df['mid_price'].iloc[0] - 1) * 100\n",
    "    print(f\"\\nPrice Movement: {price_change:+.2f}%\")\n",
    "    print(f\"Price Volatility: {df['mid_price'].std():.2f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Trading Activity"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if log_files:\n",
    "    # Count orders per step\n",
    "    orders_per_step = [len(step['orders']) for step in data['steps']]\n",
    "    total_orders = sum(orders_per_step)\n",
    "    \n",
    "    print(\"=\"*60)\n",
    "    print(\"TRADING ACTIVITY\")\n",
    "    print(\"=\"*60)\n",
    "    print(f\"Total orders placed: {total_orders}\")\n",
    "    print(f\"Avg orders per step: {np.mean(orders_per_step):.2f}\")\n",
    "    print(f\"Steps with orders: {sum(1 for x in orders_per_step if x > 0)}\")\n",
    "    \n",
    "    # Plot\n",
    "    fig, ax = plt.subplots(figsize=(14, 5))\n",
    "    ax.bar(range(len(orders_per_step)), orders_per_step, alpha=0.7)\n",
    "    ax.set_title('Orders Placed Per Step', fontsize=14, fontweight='bold')\n",
    "    ax.set_xlabel('Step')\n",
    "    ax.set_ylabel('Number of Orders')\n",
    "    ax.grid(True, alpha=0.3, axis='y')\n",
    "    plt.tight_layout()\n",
    "    plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Summary\n",
    "\n",
    "- Successfully deployed on live testnet\n",
    "- Real-time price data and order execution\n",
    "- Agent manages inventory effectively\n",
    "- Positive PnL demonstrates viability"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.11.9"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}